# Putting it all together (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

Собираем все вместе
Ранее большая часть работы выполнялась вручную (изучили, как работают токенизаторы, рассмотрели токенизацию, преобразование в идентификаторы ввода, заполнение, усечение и маски внимания).  
Однако, API Transformers 🤗 может справиться со всем этим  с помощью высокоуровневой функции, в которую мы здесь углубимся. Когда вы вызываете токенизатор непосредственно в предложении, вы получаете обратно входные данные, готовые к передаче через вашу модель:

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [2]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [3]:
model_inputs

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Здесь переменная model_inputs содержит все необходимое для корректной работы модели. Для DistilBERT это включает идентификаторы входов, а также маску внимания. Другие модели, которые принимают дополнительные входы, также будут иметь те, которые выводятся объектом токенизатора.  
Во-первых, этот метод может токенизировать одну последовательность:

In [4]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
model_inputs = tokenizer(sequences)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


Его можно наполнять в соответствии с несколькими целями:

In [ ]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

In [ ]:
# Will pad the sequences up to the model max length (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

In [ ]:
# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

Объект токенизатора может обрабатывать преобразование в определенные тензоры фреймворка, которые затем могут быть напрямую отправлены в модель.

In [ ]:
# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

Специальные токены
Если мы посмотрим на входные идентификаторы, возвращаемые токенизатором, мы увидим, что они немного отличаются от того, что у нас было ранее:

In [5]:
model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


Один идентификатор токена был добавлен в начале, а другой в конце. Давайте расшифруем две последовательности идентификаторов выше, чтобы понять, о чем идет речь:

In [6]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


Токенизатор добавил специальное слово [CLS] в начале и специальное слово [SEP] в конце. Это потому, что модель была предварительно обучена с ними, поэтому, чтобы получить те же результаты для вывода, нам нужно добавить их тоже. Обратите внимание, что некоторые модели не добавляют специальные слова или добавляют другие; модели также могут добавлять эти специальные слова только в начале или только в конце. В любом случае токенизатор знает, какие из них ожидаются, и разберется с этим за вас.

Подведение итогов: от токенизатора к модели
Теперь, когда мы увидели все отдельные шаги, которые использует объект токенизатора при применении к текстам, давайте в последний раз посмотрим, как он может обрабатывать несколько последовательностей (дополнение!), очень длинные последовательности (усечение!) и несколько типов тензоров с помощью своего основного API:

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [9]:
output.logits

tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>)

In [11]:
import torch

predictions = torch.nn.functional.softmax(output.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [5.3534e-04, 9.9946e-01]], grad_fn=<SoftmaxBackward0>)
